In [1]:
import pandas as pd
df = pd.read_csv('/kaggle/input/parseddata/processed_log_data.csv')

In [2]:
df['url'].head().to_dict()

{0: '/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53',
 1: '/image/60844/productModel/200x200',
 2: '/image/61474/productModel/200x200',
 3: '/image/14925/productModel/100x100',
 4: '/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%D8%B1-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C-%D9%BE%D8%B1%D9%86%D8%B3%D9%84%DB%8C-%D9%85%D8%AF%D9%84-PR257AT'}

In [3]:
import pandas as pd
import re # Thư viện regex
from urllib.parse import urlparse, parse_qs # << QUAN TRỌNG cho urlparse và parse_qs
from scipy.stats import entropy # Cho url_entropy ở Section 1

## Feature extracted from URL

In [ ]:
import pandas as pd
from urllib.parse import urlparse, parse_qs
from scipy.stats import entropy 

# 1. url length
df['url_len'] = df['url'].astype(str).apply(len)

# 2. Độ sâu của đường dẫn (số lượng dấu '/' trong path)
# urlparse(url_string).path sẽ lấy phần path của URL
# astype(str) để xử lý các giá trị NaN nếu có, chúng sẽ trở thành chuỗi 'nan'
df['url_path_depth'] = df['url'].astype(str).apply(lambda x: urlparse(x).path.count('/'))

# 3. Số lượng tham số query
# urlparse(url_string).query sẽ lấy phần query string
# parse_qs(...) sẽ parse query string thành dictionary
# astype(str) để xử lý NaN
df['url_query_count'] = df['url'].astype(str).apply(lambda x: len(parse_qs(urlparse(x).query)))

# 4. Entropy của URL (đo độ ngẫu nhiên/phức tạp của chuỗi URL)
# Cần xử lý trường hợp chuỗi rỗng hoặc rất ngắn có thể gây lỗi hoặc kết quả không có ý nghĩa cho entropy
def calculate_entropy(text_string):
    # Đảm bảo text_string là một chuỗi
    text = str(text_string) 
    if not text or len(text) < 2: # Entropy không có ý nghĩa với chuỗi quá ngắn hoặc rỗng
        return 0.0
    
    # Tính tần suất xuất hiện của mỗi ký tự
    prob = [float(text.count(c)) / len(text) for c in dict.fromkeys(list(text))]
    
    # Sử dụng scipy.stats.entropy
    return entropy(prob, base=2) # base=2 để tính entropy theo bit

df['url_entropy'] = df['url'].astype(str).apply(calculate_entropy)

print("\n--- Features từ URL (Section 1) đã được tạo ---")
# Hiển thị một vài dòng với các feature mới và cột url gốc để đối chiếu
# Lấy các cột có thể không tồn tại bằng df.get, nếu không sẽ gây lỗi nếu cột chưa có
columns_to_show = ['url', 'url_len', 'url_path_depth', 'url_query_count', 'url_entropy']
existing_columns_to_show = [col for col in columns_to_show if col in df.columns]
if existing_columns_to_show:
     print(df[existing_columns_to_show].head())
else:
    print("Không có cột nào để hiển thị, có thể có lỗi ở bước tạo feature.")

# Kiểm tra các giá trị NaN trong các cột mới tạo (nếu có)
new_feature_cols_s1 = ['url_len', 'url_path_depth', 'url_query_count', 'url_entropy']
for col in new_feature_cols_s1:
    if col in df.columns: # Kiểm tra xem cột đã được tạo chưa
        print(f"Số giá trị NaN trong cột '{col}': {df[col].isnull().sum()}")
    else:
        print(f"Cột '{col}' chưa được tạo.")

In [ ]:
#Feature extraction from URL
sql_injection_pattern = r"(union\s*select|select\s*.*\s*from|insert\s*into|delete\s*from|drop\s*table|xp_cmdshell|exec\s*\(|'select|--|#)"
xss_pattern = r"(<script>|javascript:|onerror\s*=|<img\s*src\s*=\s*x onerror)"
path_traversal_pattern = r"(\.\./|\.\.\\|%2e%2e%2f|%2e%2e%5c)"
# LFI/RFI pattern cần cẩn thận hơn, vì nó có thể khớp với các URL bình thường nếu áp dụng rộng rãi
# Có lẽ nên tìm các scheme này khi chúng xuất hiện ở những vị trí không mong muốn, ví dụ, giá trị của một tham số.
# Nếu cột 'url' của bạn không có query string dạng key=value, việc tìm LFI/RFI trong query sẽ không có tác dụng.
# Chúng ta có thể tìm các mẫu này trực tiếp trong path, nhưng cần cẩn thận với false positives.
lfi_rfi_path_pattern = r"(file:\/\/|php:\/\/input|php:\/\/filter)" 

df['url_has_sqli'] = df['url'].astype(str).apply(lambda x: 1 if re.search(sql_injection_pattern, x, re.IGNORECASE) else 0)
df['url_has_xss'] = df['url'].astype(str).apply(lambda x: 1 if re.search(xss_pattern, x, re.IGNORECASE) else 0)
df['url_has_path_traversal'] = df['url'].astype(str).apply(lambda x: 1 if re.search(path_traversal_pattern, x) else 0)

# Nếu df['url'] không có query string dạng ?key=value, feature sau sẽ không có nhiều ý nghĩa.
# Nếu bạn muốn tìm LFI/RFI trong path, bạn có thể làm:
df['url_path_has_lfi_rfi_pattern'] = df['url'].astype(str).apply(lambda x: 1 if re.search(lfi_rfi_path_pattern, urlparse(x).path, re.IGNORECASE) else 0)


def get_file_extension(url_path_string): # Đổi tên biến để rõ ràng hơn
    # Giả sử url_path_string là phần path, có thể có query string hoặc không
    path_component = urlparse(url_path_string).path
    if '.' in path_component:
        filename = path_component.split('/')[-1] # Lấy phần cuối sau dấu /
        if '.' in filename: # Đảm bảo có dấu chấm trong tên file thực sự
             return filename.split('.')[-1].lower()
    return ''

df['url_file_extension'] = df['url'].astype(str).apply(get_file_extension)

common_script_extensions = ['php', 'asp', 'aspx', 'jsp', 'sh', 'bash', 'py', 'exe', 'dll']
df['url_is_script_ext'] = df['url_file_extension'].apply(lambda x: 1 if x in common_script_extensions else 0)

print("\n--- Features từ URL - Mẫu đáng ngờ và Phần mở rộng (Đã sửa) ---")
print(df[['url', 'url_has_sqli', 'url_has_xss', 'url_has_path_traversal', 'url_path_has_lfi_rfi_pattern', 'url_file_extension', 'url_is_script_ext']].head())

In [ ]:
# Test với các URL có phần mở rộng
test_urls_with_ext = ['/path/to/file.php', '/another/script.asp?id=1', '/static/image.Jpg', '/no_extension_here/', '/archive.tar.gz']
for test_url in test_urls_with_ext:
    ext = get_file_extension(test_url) # Giả sử hàm get_file_extension đã được định nghĩa
    is_script = 1 if ext in common_script_extensions else 0 # Giả sử common_script_extensions đã được định nghĩa
    print(f"URL: {test_url}, Extension: '{ext}', IsScript: {is_script}")

In [ ]:
df

In [ ]:
print("\nPhân phối các phần mở rộng file (Top 20):")
print(df['url_file_extension'].value_counts().nlargest(20))
print(f"\nSố dòng có url_is_script_ext = 1: {df['url_is_script_ext'].sum()}")

In [ ]:
pip install user-agents ua-parser pyyaml

## Feature extracted from user agent

In [ ]:
from user_agents import parse

# Parse User-Agent string
# .astype(str) để xử lý NaN, chúng sẽ được parse như một chuỗi 'nan'
# và thư viện user-agents sẽ trả về một UserAgent object mặc định (generic) cho các chuỗi không hợp lệ.
df['ua_parsed'] = df['user_agent'].astype(str).apply(parse)

# 1. Trình duyệt (Browser Family)
df['ua_browser_family'] = df['ua_parsed'].apply(lambda ua: ua.browser.family)

# 2. Phiên bản trình duyệt (Major Version)
df['ua_browser_version_major'] = df['ua_parsed'].apply(lambda ua: ua.browser.version[0] if ua.browser.version else None)

# 3. Hệ điều hành (OS Family)
df['ua_os_family'] = df['ua_parsed'].apply(lambda ua: ua.os.family)

# 4. Phiên bản hệ điều hành (Major Version)
df['ua_os_version_major'] = df['ua_parsed'].apply(lambda ua: ua.os.version[0] if ua.os.version else None)

# 5. Thiết bị (Device Family/Brand)
df['ua_device_family'] = df['ua_parsed'].apply(lambda ua: ua.device.family)
df['ua_device_brand'] = df['ua_parsed'].apply(lambda ua: ua.device.brand)

# 6. Có phải là Bot không?
df['ua_is_bot'] = df['ua_parsed'].apply(lambda ua: 1 if ua.is_bot else 0)

# 7. Có phải là thiết bị di động không?
df['ua_is_mobile'] = df['ua_parsed'].apply(lambda ua: 1 if ua.is_mobile else 0)

# 8. Có phải là máy tính bảng không?
df['ua_is_tablet'] = df['ua_parsed'].apply(lambda ua: 1 if ua.is_tablet else 0)

# 9. Có phải là PC không?
df['ua_is_pc'] = df['ua_parsed'].apply(lambda ua: 1 if ua.is_pc else 0)

# 10. Có phải là thiết bị cảm ứng không? (Thường trùng với mobile/tablet)
df['ua_is_touch_capable'] = df['ua_parsed'].apply(lambda ua: 1 if ua.is_touch_capable else 0)

# 11. Độ dài chuỗi User-Agent gốc
df['ua_length'] = df['user_agent'].astype(str).apply(len)

# 12. Entropy của chuỗi User-Agent gốc (sử dụng lại hàm calculate_entropy từ Section 1)
# def calculate_entropy(text_string): ... (Đảm bảo hàm này đã được định nghĩa)
df['ua_entropy'] = df['user_agent'].astype(str).apply(calculate_entropy)


# Hiển thị một vài dòng với các feature mới
cols_to_show_ua = [
    'user_agent', 'ua_browser_family', 'ua_os_family', 'ua_device_brand', 
    'ua_is_bot', 'ua_is_mobile', 'ua_is_pc', 'ua_length', 'ua_entropy'
]
# Lọc ra các cột thực sự tồn tại trong df để tránh lỗi nếu có cột nào đó không được tạo
existing_cols_ua = [col for col in cols_to_show_ua if col in df.columns]
if existing_cols_ua:
    print("\n--- Features từ User-Agent ---")
    print(df[existing_cols_ua].head())

# Xóa cột ua_parsed trung gian nếu muốn (nó chứa object, tốn bộ nhớ)
df.drop('ua_parsed', axis=1, inplace=True)

## Feature extraction from status code

In [ ]:
# --- Giả sử df đã có cột 'status' (int64) và 'size' (int64) ---

# 1. Status code có phải là lỗi Client không (4xx)?
df['status_is_client_error'] = df['status'].apply(lambda x: 1 if 400 <= x < 500 else 0)

# 2. Status code có phải là lỗi Server không (5xx)?
df['status_is_server_error'] = df['status'].apply(lambda x: 1 if 500 <= x < 600 else 0)

# 3. Status code có phải là lỗi nói chung không (4xx hoặc 5xx)?
df['status_is_error'] = ((df['status_is_client_error'] == 1) | (df['status_is_server_error'] == 1)).astype(int)

# 4. Status code có phải là thành công không (2xx)?
df['status_is_success'] = df['status'].apply(lambda x: 1 if 200 <= x < 300 else 0)

# 5. Status code có phải là chuyển hướng không (3xx)?
df['status_is_redirect'] = df['status'].apply(lambda x: 1 if 300 <= x < 400 else 0)

# 6. Response size có bằng 0 không?
df['size_is_zero'] = df['size'].apply(lambda x: 1 if x == 0 else 0)

# (Chúng ta đã có 'size' trực tiếp, có thể dùng nó cho mô hình.
#  Nếu muốn, có thể tạo thêm các bin cho size, vd: size_small, size_medium, size_large
#  nhưng việc này phụ thuộc vào phân phối của size trong dữ liệu của bạn)

cols_to_show_status_size = [
    'status', 'size', 'status_is_client_error', 'status_is_server_error',
    'status_is_success', 'status_is_redirect', 'size_is_zero'
]
existing_cols_ss = [col for col in cols_to_show_status_size if col in df.columns]
if existing_cols_ss:
    print("\n--- Features từ Status Code và Size ---")
    print(df[existing_cols_ss].head())

## Feature extraction from referrer

In [ ]:
# --- Giả sử df đã có cột 'referrer' (object/string) ---
# và hàm calculate_entropy đã được định nghĩa

# 1. Độ dài Referrer
# .astype(str) để xử lý NaN, chúng sẽ trở thành chuỗi 'nan'
df['referrer_len'] = df['referrer'].astype(str).apply(len)

# 2. Entropy của Referrer
df['referrer_entropy'] = df['referrer'].astype(str).apply(calculate_entropy)

# 3. Referrer có rỗng không (thường được biểu thị bằng '-' hoặc chuỗi rỗng)?
df['referrer_is_empty'] = df['referrer'].apply(lambda x: 1 if x == '-' or pd.isna(x) or str(x).strip() == '' else 0)

# 4. Trích xuất domain của referrer (nếu có)
def get_referrer_domain(referrer_string):
    ref_str = str(referrer_string)
    if pd.isna(referrer_string) or ref_str == '-' or not ref_str.startswith('http'):
        return 'none' # Hoặc 'empty', 'internal' tùy theo cách bạn muốn xử lý
    try:
        # urlparse cần URL đầy đủ, nếu referrer chỉ là path thì sẽ không có netloc
        parsed_ref = urlparse(ref_str)
        if parsed_ref.netloc:
            return parsed_ref.netloc.lower()
        else: # Có thể là một path nội bộ hoặc một URL không chuẩn
            return 'unknown_format_but_not_empty'
    except Exception: # Bắt các lỗi parsing URL không mong muốn
        return 'parse_error'

df['referrer_domain'] = df['referrer'].apply(get_referrer_domain)

# 5. Referrer có phải là từ một domain khác với domain của trang web bạn đang phân tích không?
#    (Cần biết domain của trang web bạn, ví dụ: 'yourwebsite.com')
#    Nếu không biết, có thể so sánh với các domain phổ biến nhất trong dữ liệu referrer.
#    Ở đây, chúng ta sẽ tạo một feature đơn giản là "referrer không phải là none/empty"
df['referrer_is_external_or_valid'] = df['referrer_domain'].apply(lambda x: 0 if x in ['none', 'parse_error', 'unknown_format_but_not_empty'] else 1)


cols_to_show_ref = [
    'referrer', 'referrer_len', 'referrer_entropy', 
    'referrer_is_empty', 'referrer_domain', 'referrer_is_external_or_valid'
]
existing_cols_ref = [col for col in cols_to_show_ref if col in df.columns]
if existing_cols_ref:
    print("\n--- Features từ Referrer ---")
    print(df[existing_cols_ref].head())

## Behavioral/Frequency features

In [ ]:
# --- Giả sử df đã có cột 'ip', 'event_id_method_url', 'status_is_error', 'url_query_count' ---

# 1. Số lượng request từ một IP
df['ip_request_count_total'] = df.groupby('ip')['ip'].transform('count')

# 2. Số lượng event_id (loại request)ユニーク mà một IP đã thực hiện
df['ip_unique_event_id_count'] = df.groupby('ip')['event_id_method_url'].transform('nunique')

# 3. Tỷ lệ request lỗi (status_is_error=1) của một IP
# Cần cẩn thận chia cho 0 nếu một IP không có request nào (không xảy ra nếu groupby 'ip')
# Hoặc nếu ip_request_count_total được tính trước và có thể bằng 0 (không thể)
df['ip_error_rate'] = df.groupby('ip')['status_is_error'].transform('mean') # mean của 0 và 1 chính là tỷ lệ

# 4. Tần suất trung bình của các event_id mà một IP truy cập
# (event_id_method_url_freq_global)
# Tính tần suất toàn cục của mỗi event_id trước
event_id_global_freq = df['event_id_method_url'].value_counts(normalize=True)
df['event_id_global_freq'] = df['event_id_method_url'].map(event_id_global_freq)

# 5. Entropy của các event_id mà một IP đã truy cập
# Tính toán này phức tạp hơn và thường cần một hàm tùy chỉnh với groupby().apply()
# Đây là một ví dụ đơn giản hóa: độ hiếm trung bình của các event_id mà IP đó truy cập
df['ip_avg_event_id_rarity'] = df.groupby('ip')['event_id_global_freq'].transform('mean') 
# Nếu giá trị này thấp, nghĩa là IP đó thường xuyên truy cập các event_id hiếm (1/freq cao)


# 6. Số lượng tham số query trung bình cho mỗi IP
df['ip_avg_query_count'] = df.groupby('ip')['url_query_count'].transform('mean')


cols_to_show_ip_behavior = [
    'ip', 'ip_request_count_total', 'ip_unique_event_id_count', 'ip_error_rate',
    'event_id_global_freq', 'ip_avg_event_id_rarity', 'ip_avg_query_count'
]
existing_cols_ip_b = [col for col in cols_to_show_ip_behavior if col in df.columns]
if existing_cols_ip_b:
    print("\n--- Features Hành vi theo IP (Toàn cục) ---")
    print(df[existing_cols_ip_b].head())

## Time-based feature

In [ ]:
# --- Giả sử df đã có cột 'timestamp_dt' (đã parse sang datetime) ---

# 1. Giờ trong ngày (0-23)
df['hour_of_day'] = df['timestamp_dt'].dt.hour

# 2. Ngày trong tuần (0=Thứ Hai, 6=Chủ Nhật)
df['day_of_week'] = df['timestamp_dt'].dt.dayofweek # Monday=0, Sunday=6

# 3. Có phải cuối tuần không? (Thứ 7 hoặc Chủ Nhật)
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# 4. Phần của ngày (ví dụ: Sáng, Chiều, Tối, Đêm)
def get_part_of_day(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'
df['part_of_day'] = df['hour_of_day'].apply(get_part_of_day)


cols_to_show_time = [
    'timestamp_dt', 'hour_of_day', 'day_of_week', 'is_weekend', 'part_of_day'
]
existing_cols_time = [col for col in cols_to_show_time if col in df.columns]
if existing_cols_time:
    print("\n--- Features theo Thời gian ---")
    print(df[existing_cols_time].head())